Import libraries:

In [123]:
import re
import time
import nltk
import pandas as pd
import seaborn as sns
from requests import get
from random import randint

from rouge import Rouge 

import spacy
from spacy.symbols import nsubj, nsubjpass, VERB

from pathlib import Path
from requests import get
from tika import parser


import io
import pickle
import csv

import winsound
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx as nx
from requests import get
from googlesearch import search
from textblob import TextBlob

from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from bs4.element import Comment
from bs4 import BeautifulSoup
import urllib.request

import time
import collections
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from nltk import tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import chromedriver_binary

Import CDFs:

In [3]:
def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def scoring(a,b):
    try:
        a = a.replace(',', '').split()
        b = b.replace(',', '').split()
        score = int(100*len(set(a).intersection(b))/(len(set(a))))
    
    except:
        score = 0    
        
    return score

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_normalize(txt):
    processed_text = re.sub('[^a-zA-Z]', ' ', txt)
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]
    
    return tokens

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def create_graph(text, nodes=25, max_component=False, common=500):

    ## Normalize text
    tokens = text_normalize(text)
    ## /Normalize text

    ## create graph
    bigrams=list(nltk.ngrams(tokens, 2))
    bigram_counts = collections.Counter(bigrams)

    bigram_df = pd.DataFrame(bigram_counts.most_common(common), columns=['bigram', 'count'])
    d = bigram_df.set_index('bigram').T.to_dict('records')

    G = nx.Graph()
    for k, v in d[0].items():
            G.add_edge(k[0], k[1], weight=(v*10))
    ## /create graph

    ## show max connected component
    if max_component==True:
        G = max(connected_component_subgraphs(G), key=len)
        G = nx.dfs_tree(G)
    ## /show max connected component

    ## remove isolates
    G.remove_nodes_from(list(nx.isolates(G)))
    ## /remove isolates

    ## filter graph by degree
    threshold = int(G.number_of_nodes()*(max(nx.degree_centrality(G).values()))) #max centrality, nodes

    for i in range(0, threshold):
        F = nx.Graph() # empty graph
        fedges = filter(lambda x: G.degree()[x[0]] > 0 and G.degree()[x[1]] > i, G.edges())
        F.add_edges_from(fedges)
        F.remove_nodes_from(list(nx.isolates(F)))

        if F.number_of_nodes() < nodes:
            break
    ## /filter graph by degree

    # pos = nx.spring_layout(F, iterations=500)

    return F

def get_keys(url):
    
    ## parse url
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find_all(['p', 'article', 'section', 'title', 'h1', 'h2', 'h3'])
    rawtext = text_from_html(str(paragraphs))
    ## /parse url
    
    ## nlp processing
    tokens = text_normalize(rawtext)
    ## /nlp processing

    ## Bigrams
    keys_list_bg = []
    keys_list_gr = []
    corpus = tokenize.sent_tokenize(' '.join(tokens))
    
    if len(corpus) > 0:
        vec = CountVectorizer(ngram_range=(2,2), max_features=100).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0)
        
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
        
        keys_list_bg = [str(i[0]) for i in words_freq[:10]]
        keys_list_bg = list(set(keys_list_bg))
        
        F = create_graph(rawtext)
        d = dict(F.degree)
        keys_list_gr = [i[0] for i in sorted(F.degree, key=lambda x: x[1], reverse=True)][:5]
    ## /Bigrams
    
    # mix keys
    keys_list = (keys_list_gr + keys_list_bg)
    keys_cap = (''.join([i.capitalize() for i in ' '.join(keys_list).split()])).split()
    # /mix keys

    keys = ', '.join(keys_list)
    
    return keys, rawtext

def sumy_summary(docx,n):
    parser = PlaintextParser.from_string(docx,Tokenizer("english"))
    lex_summarizer = LexRankSummarizer()
    summary = lex_summarizer(parser.document,n)
    summary_list = [str(sentence) for sentence in summary]
    result = ' '.join(summary_list)
    return result

def syntax_full(spacy_sentence):
    result=[] #list of verbs who have nsubj or nsubjpass dependants
    for token in spacy_sentence:
        #check if token is nsubj-type dependant of a verb:
        if (token.dep == nsubj or token.dep == nsubjpass) and token.head.pos == VERB:
            result.append(token.head)
    if result:
        return True
    else:
        return False
    
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def get_jaccard_sim(x, y): 
    a = set(x.split()) 
    b = set(y.split())
    c = list(a.intersection(b))
    return float(len(c)) / (len(list(a)) + len(list(b)) - len(list(c)))

### Parse Arxiv to abstracts and urls:

Create URLs:

In [4]:
request_list = ['natural language processing', 'text summarization nlp', 'keywords extraction', 'graph analysis nlp']
request_list = request_list + ['text neural network', 'abstractive summarization', 'text lstm', 'text graph analysis']
request_list = request_list + ['text seq to seq', 'text bert', 'gensim summarization', 'tensorflow text summarization']
request_list = request_list + ['website summarizer', 'nlp text representation', 'text clustering']
request_list = request_list + ['nlp chatbot', 'nlp twitter trends', 'deep learning']

Get urls:

In [ ]:
urls = []

for j in tqdm(request_list[:1]):
    print('\nRequest: '+str(j))
    for i in tqdm(range(0,50,50)):
        hrefs = []
        try:
            htmlString = get('https://arxiv.org/search/?query=' + str(j) + '&searchtype=all&source=header&start=' + str(i))
            soup = BeautifulSoup(htmlString.content, 'html5lib') 
            hrefs = soup.find_all('a', {'href': re.compile(r'arxiv.org/pdf/')})
            
            if (len(hrefs) > 0):
                urls.append(i['href']) for i in hrefs
        except:
            print ('Connection error')  
    print('Number of inique urls: '+ str(len(set(urls))))    

Number of unique urls:

In [ ]:
winsound.Beep(2500, 1000)
print('Number of inique urls: '+ str(len(set(urls))))

Write urls:

In [ ]:
df = pd.DataFrame(list(set(urls)), columns=['urls'])
df.drop_duplicates(subset = 'urls', keep = 'first', inplace = True)
df = df.replace('\n','', regex=True)
df.to_excel (r'datasets/sets_arxiv/urls.xlsx', index = False, header=True)
df.head()

### Get abstracts, summaries and scores

Read urls:

In [64]:
df = pd.read_excel(f'datasets/sets_arxiv/urls.xlsx', sheet_name='Sheet1')
df['abstracts'] = df['abstracts'].apply(lambda x: BeautifulSoup(x, 'lxml').text)
df.drop_duplicates(subset = 'urls', keep = 'first', inplace = True)
df = df.replace('\n','', regex=True)
df.head()

,urls,abstracts
0,https://arxiv.org/pdf/1909.00430,Accurate predictions for the non-linea...
1,https://arxiv.org/pdf/1910.00194,Different layers of deep convolutional...
2,https://arxiv.org/pdf/2007.00263,Supervised relation extraction methods...
3,https://arxiv.org/pdf/1811.04076,Predicting discharge medications right...
4,https://arxiv.org/pdf/1911.01196,It is often suggested that a super mas...


Create list of urls:

In [65]:
urls = list(df['urls']) 

Abstract scoring:

In [147]:
SCROLL_PAUSE_TIME = 8
ERROR_NUM = 0
CONNECT_NUM = 0

k = 0
for item in tqdm(urls[:2000]): #change the start point      
    uris = []
    scores = []
    abstracts = []
    ml_abstracts = []
    
    k = k+1   
    print(item)
    print('Number of urls:', int(k))
    
    time.sleep(randint(1,SCROLL_PAUSE_TIME))   
    CONNECT_NUM = CONNECT_NUM + 1   
            
    try:
        # handmade abstract
        htmlString = get('https://arxiv.org/search/?query=' + str(item))
        soup = BeautifulSoup(htmlString.content, 'html5lib')
        abstract = soup.find('blockquote').text.replace('\n','')
        # /handmade abstract
        
        # ml abstract - parse text
        file_data = parser.from_file(item)
        rawtext = file_data['content'].replace('\n','')
        rawtext = re.sub("[^.,';:!?a-zA-Z0-9 \n\.]", "", rawtext)
        # /ml abstract - parse text
             
        # ml abstract - remove handmade abstract       
        abstract_list = nltk.tokenize.sent_tokenize(abstract)
        rawtext_list = nltk.tokenize.sent_tokenize(rawtext) 

        for i in abstract_list:
            for j in rawtext_list:
                if get_jaccard_sim(i,j) >= 0.5:
                    rawtext = rawtext.replace(str(j),'')     
        # /ml abstract - remove handmade abstract
        
        # ml abstract - syntactic filter
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(rawtext)
        valid_sents=[]
        
        for s in list(doc.sents):
            if syntax_full(s):
                valid_sents.append(s.text)

        rawtext = ' '.join(valid_sents)
        #/ml abstract - syntactic filter
               
        # ml abstract - summarize text
        sentences = (min(10,int(len(tokenize.sent_tokenize(rawtext)))) - 2)
        txt = sumy_summary(rawtext, sentences)
        txt = get_unique_text(' '.join(re.split(r'(?<=[.;])\s', rawtext)[:2] + txt.splitlines()))
        txt = txt.replace('\n','')
        # /ml abstract - summarize text

        # define score
        sim_score = round(list(list(dict(list(Rouge().get_scores(abstract, txt))[0]).values())[1].values())[0],2)
        #/define score
        
        # create lists
        uris.append(item)
        abstracts.append(abstract)
        ml_abstracts.append(txt)
        scores.append(sim_score)
        # /create lists

        # write dataframe
        df_score = pd.read_excel(f'datasets/scoring.xlsx', sheet_name='Sheet1')
        df_score = df_score.append(pd.DataFrame(zip(uris, abstracts, ml_abstracts, scores)), ignore_index=True)
        df_score.to_excel(r'datasets/scoring.xlsx', index = False, header=True)
        #/write dataframe
     
    except:
        ERROR_NUM = ERROR_NUM + 1
        winsound.Beep(2500, 1000)
        print ('Connection error')
         
print('Number of attempts:', CONNECT_NUM)
print('Number of errors:', ERROR_NUM)
winsound.Beep(2500, 1000)





  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

https://arxiv.org/pdf/1909.00430
Number of urls: 1


2020-08-12 20:19:35,551 [MainThread  ] [INFO ]  Retrieving https://arxiv.org/pdf/1909.00430 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1909.00430.
INFO:tika.tika:Retrieving https://arxiv.org/pdf/1909.00430 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1909.00430.




  0%|                                                                               | 1/2000 [00:09<5:24:53,  9.75s/it]

https://arxiv.org/pdf/1910.00194
Number of urls: 2


2020-08-12 20:19:51,206 [MainThread  ] [INFO ]  Retrieving https://arxiv.org/pdf/1910.00194 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1910.00194.
INFO:tika.tika:Retrieving https://arxiv.org/pdf/1910.00194 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1910.00194.




  0%|                                                                               | 2/2000 [00:24<6:12:12, 11.18s/it]

...

https://arxiv.org/pdf/1712.00170
Number of urls: 1999


2020-08-13 20:11:02,917 [MainThread  ] [INFO ]  Retrieving https://arxiv.org/pdf/1712.00170 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1712.00170.
INFO:tika.tika:Retrieving https://arxiv.org/pdf/1712.00170 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1712.00170.




100%|██████████████████████████████████████████████████████████████████████████▉| 1999/2000 [23:51:37<00:15, 15.29s/it]

https://arxiv.org/pdf/1801.04813
Number of urls: 2000


2020-08-13 20:11:16,828 [MainThread  ] [INFO ]  Retrieving https://arxiv.org/pdf/1801.04813 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1801.04813.
INFO:tika.tika:Retrieving https://arxiv.org/pdf/1801.04813 to C:\Users\SKAMEN~1\AppData\Local\Temp/pdf-1801.04813.




100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [23:51:52<00:00, 42.96s/it]


Number of attempts: 2000
Number of errors: 6


Remobe duplicates:

In [160]:
df_score = pd.read_excel(f'datasets/scoring.xlsx', sheet_name='Sheet1')
df_score.columns = ['url','abstract','ml_abstract','score']
df_score['score'] = df_score['score']*100
df_score['score'] = df_score['score'].astype(int)
df_score.drop_duplicates(subset = 'url', keep = 'first', inplace = True)
df_score.head()

,url,abstract,ml_abstract,score
0,https://arxiv.org/pdf/1909.00430,Abstract: Deep learning systems thrive ...,yoav.goldberggmail.comAbstractDeep learning sy...,21
1,https://arxiv.org/pdf/1910.00194,Abstract: Contextualized word represent...,gan weechungu.nus.eduAbstractContextualized wo...,13
2,https://arxiv.org/pdf/2007.00263,"Abstract: Android, being the most wides...","College of Computing Umm AlQura University, Sa...",15
3,https://arxiv.org/pdf/1811.04076,Abstract: This paper describes a method...,hojo.nobukatsulab.ntt.co.jpABSTRACTThis paper ...,24
4,https://arxiv.org/pdf/1911.01196,Abstract: Unsupervised text embedding h...,chaozhanggatech.edu 3 lance.m.kaplan.civmail.m...,16


Write to xlsx:

In [161]:
df_score.to_excel (r'datasets/scoring.xlsx', index = False, header=True)

### Statistics

Read xlsx:

In [162]:
df_score = pd.read_excel(f'datasets/scoring.xlsx', sheet_name='Sheet1')

Define score:

In [163]:
len(df_score)

1994

Statistics:

In [165]:
print('Median score:', int(np.median(df_score['score'])), '%')
print('Standard deviation:', int(np.std(df_score['score'])), '%')
print('Max/min scattering:', int((np.max(df_score['score'])-np.min(df_score['score']))/2), '%')
print('Number of records:', len(df_score))

Median score: 12 %
Standard deviation: 5 %
Max/min scattering: 17 %
Number of records: 1994
